In [104]:
import pandas as pd
import numpy as np

### Extraction of languages

In [105]:
train = pd.read_csv('dialect-identification/data/raw/dslcc4/DSL-TRAIN.TXT', sep='\t', header=None)
test = pd.read_csv('dialect-identification/data/raw/dslcc4/DSL-TEST-GOLD.TXT', sep='\t', header=None)
dev = pd.read_csv('dialect-identification/data/raw/dslcc4/DSL-DEV.TXT', sep='\t', header=None)

sets = [train, test, dev]

In [106]:
keep = ['es-ES', 'sr', 'hr', 'bs', 'es-PE', 'es-AR' , 'pt-PT', 'pt-BR']

for i in range(len(sets)):
    sets[i] = sets[i][sets[i][1].isin(keep)]


In [107]:
def assign_language_group(row):
    """
    Assigns new column "group" to a given language.

    Parameters
    ----------
    row : pandas.Series
        A row of the DataFrame.
    
    Returns
    -------
    str
        The group of the language.

    """
    if row[1] in ['es-ES', 'es-PE', 'es-AR']:
        return 'es'
    elif row[1] in ['pt-PT', 'pt-BR']:
        return 'pt'
    elif row[1] in ['hr', 'bs', 'sr']:
        return 'bks'
        
for i in range(len(sets)):
    sets[i]['group'] = sets[i].apply(assign_language_group, axis=1)


/var/folders/8v/5_c7kn_13jsgmgqz56r3yx440000gp/T/ipykernel_8456/2141305198.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sets[i]['group'] = sets[i].apply(assign_language_group, axis=1)
/var/folders/8v/5_c7kn_13jsgmgqz56r3yx440000gp/T/ipykernel_8456/2141305198.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sets[i]['group'] = sets[i].apply(assign_language_group, axis=1)
/var/folders/8v/5_c7kn_13jsgmgqz56r3yx440000gp/T/ipykernel_8456/2141305198.py:24: SettingWithCopyWarning: 
A value is trying to 

### Adding german dialects from a different data set

In [108]:
train_swiss = pd.read_csv('dialect-identification/data/swiss-dialects/train.txt', sep='\t', header=None)
dev_swiss = pd.read_csv('dialect-identification/data/swiss-dialects/dev.txt', sep='\t', header=None)
train_swiss['group'] = 'GER'
dev_swiss['group'] = 'GER'

In [109]:
train = pd.concat([sets[0], train_swiss])
test = pd.concat([sets[2], dev_swiss])
train = pd.concat([train,sets[1]])


In [110]:
#train.to_csv('data/all/train.csv', index=False)
#test.to_csv('data/all/test.csv', index=False)

## Preprocessing

In [111]:
train_prepro = train.copy()
test_prepro = test.copy()
sets = [train_prepro, test_prepro]

In [112]:
import string
import re
import random

def preprocess_text(text):
    """
    Preprocesses a text by removing all unnecessary characters. Keep characters over letters though and do not lowercase.
    """

    text = text.strip()
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    text = re.sub(r"http\S+", "", text)
    #text = re.sub("[^A-Za-z]+", " ", text)
    text = re.sub("[^\w]+", " ", text)
    text = re.sub(r'(\d+)', ' ', text)
    text = re.sub(r'(\s+)', ' ', text)
    return text


In [113]:
for i in range(len(sets)):
    sets[i][0] = sets[i][0].apply(preprocess_text)

In [114]:
test_prepro['group'].value_counts()

group
bks    5916
es     5747
GER    4530
pt     3923
Name: count, dtype: int64

In [115]:
train_prepro.rename(columns={'group':'GROUP', 1:'VARIETY', 0:'TEXT'}, inplace=True)
test_prepro.rename(columns={'group':'GROUP', 1:'VARIETY', 0:'TEXT'}, inplace=True)

train_prepro.to_csv('dialect-identification/data/all/train_prepro.csv', index=False)
test_prepro.to_csv('dialect-identification/data/all/test_prepro.csv', index=False)
train_prepro.tail()

,TEXT,VARIETY,GROUP
13345,Projektant Peković izvijestio je gradonačelnik...,hr,bks
13346,As declarações de Juppé foram posteriormente d...,pt-PT,pt
13347,La puesta en marcha de Lanbide es consecuencia...,es-ES,es
13349,La Asociación Brasileña de la Industria Produc...,es-AR,es
13351,Premijer i ministar unutrašnjih poslova Ivica ...,bs,bks
